In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import warnings
import json
import re
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

In [13]:
json_file = 'cities.json'

with open(json_file, 'r') as f:
    md = json.load(f)

In [14]:
#need to add info for non-WWC cities

cities = pd.read_csv('../../../WWC_basics.csv')

In [15]:
cs = cities['City, State'].str.split(",", expand = True)
cities['city'] = cs[0] + ' city'

In [17]:
cities.tail()

,"City, State",State,Pol_Lean_State,Region,Population 2015,Population,Median Income 2015,Median Income,%White 2015,% Non-White 2010 (unless noted),% White Using 2015 Data and 2010 Data if 2015 Not Available,% White,Budget_1617,Governance,Mayor,City Manager,Exec_Party,Next Election Date,City Priorities,City Priorities Source,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Lines of Work,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Focus Area,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,City POC Primary,POC Primary Email,Effective Speaker (POC),City POC Secondary,POC Secondary Email,Effective Speaker (POC).1,RFA Lead,Notes,"City, State.1",Unnamed: 77,City,city
128,"Sacramento, CA",CA,Solid Democratic,Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,978400000,Council-Manager,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento city
129,"San Mateo, CA",CA,Solid Democratic,Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293300000,Council-Manager,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo city
130,"Tukwila, WA",WA,Lean Democratic,Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170609147,Mayor-Council,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tukwila city
131,"Vallejo, CA",CA,Solid Democratic,Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206213579,Council-Manager,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vallejo city
132,"West Sacramento, CA",CA,Solid Democratic,Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105358000,Council-Manager,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Sacramento city


In [18]:
cities = cities[['city', 'State', 'Pol_Lean_State', 'Region', 'Governance', 'Budget_1617', 'Exec_Party']]

In [8]:
df = pd.read_csv('/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/Riverside.csv')

In [10]:
k = 'Albuquerque city'

In [1]:

md = {"Riverside city" : {
 	"state": "CA",
 	"wwc": "Y",
 	"portal_type": "GQ",
	"odp":"Y",
	"odp_launch": "03-07-2017",
	"portal": "Y",
	"robust_portal": "N",
	"portal_launch":"01-01-2014" 
	}}

In [19]:
for key, value in md.items():
    city = re.findall(r'([^\s]+)', key)[0]
    filepath = '/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city)
    try:
        df = pd.read_csv(filepath)
    except:
        pass
    print(df.columns)
    df['Create Date'] = pd.to_datetime(df['Create Date'])
    df['month_year'] = df['Create Date'].dt.to_period('M')
    month_count = df.groupby('month_year').count()
    month_count.rename(index=str, columns={"Create Date": "count"}, inplace = True)
    month_count.reset_index(inplace=True)
    print(month_count.head())
    mc = month_count[['month_year', 'count']]
    mc['city'] = key
    mc['state'] = value['state']
    if value['odp'] == "Y":
        odp_dl = value['odp_launch'].split('-')
        mc['odp_date'] = dt.datetime(int(odp_dl[2]),int(odp_dl[0]),int(odp_dl[1]))
        mc['policy'] = np.where(pd.to_datetime(mc['month_year'])< mc['odp_date'], 0, 1)
    else:
        mc['odp_date'] = 'NaN'
        mc['policy'] = 0
        
    if value['portal'] == "Y":
        portal_dl = value['portal_launch'].split('-')
        mc['portal_date'] = dt.datetime(int(portal_dl[2]),int(portal_dl[0]),int(portal_dl[1]))
        mc['portal'] = np.where(pd.to_datetime(mc['month_year'])< mc['portal_date'], 0, 1)
    else:
        mc['portal_date'] = 'NaN'
        mc['portal'] = 0
        
    mc['year'] = pd.to_datetime(mc['month_year']).dt.year
    
    first_month = min(df['month_year'])
    
    mc['months'] = ((pd.to_datetime(mc['month_year']).dt.year - first_month.year) * 12 + 
            pd.to_datetime(mc['month_year']).dt.month - first_month.month)

    mc['2015'] = np.where(mc['year']== 2015, 1, 0)
    mc['2016'] = np.where(mc['year']== 2016, 1, 0)
    mc['2017'] = np.where(mc['year']== 2017, 1, 0)
    mc['2018'] = np.where(mc['year']== 2018, 1, 0)
        
        

FileNotFoundError: File b'/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/Albuquerque.csv' does not exist

In [12]:
mc

,month_year,count,city,state,odp_date,policy,portal_date,portal,year,months,2015,2016,2017,2018
0,2015-12,13,Riverside city,CA,2017-03-07,0,2014-01-01,1,2015,0,1,0,0,0
1,2016-01,47,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,1,0,1,0,0
2,2016-02,49,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,2,0,1,0,0
3,2016-03,43,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,3,0,1,0,0
4,2016-04,39,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,4,0,1,0,0
5,2016-05,41,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,5,0,1,0,0
6,2016-06,38,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,6,0,1,0,0
7,2016-07,56,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,7,0,1,0,0
8,2016-08,51,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,8,0,1,0,0
9,2016-09,36,Riverside city,CA,2017-03-07,0,2014-01-01,1,2016,9,0,1,0,0


In [10]:
df['Create Date'] = pd.to_datetime(df['Create Date'])

In [11]:
df['month_year'] = df['Create Date'].dt.to_period('M')

In [12]:
min(df['month_year'])

Period('2015-12', 'M')

In [50]:
month_count = df.groupby('month_year').count()

In [51]:
month_count.rename(index=str, columns={"Create Date": "count"}, inplace = True)

In [52]:
month_count.reset_index(inplace=True)

In [53]:
mc = month_count[['month_year', 'count']]

In [54]:
mc['city'] = 'Riverside city'

In [55]:
mc['state'] = 'CA'

In [56]:
mc['odp_date'] = dt.datetime(2017,3,7)
mc['year'] = pd.to_datetime(mc['month_year']).dt.year

In [57]:
mc.head()

,month_year,count,city,state,odp_date,year
0,2015-12,13,Riverside city,CA,2017-03-07,2015
1,2016-01,47,Riverside city,CA,2017-03-07,2016
2,2016-02,49,Riverside city,CA,2017-03-07,2016
3,2016-03,43,Riverside city,CA,2017-03-07,2016
4,2016-04,39,Riverside city,CA,2017-03-07,2016


In [58]:
#mc['time']= (pd.to_datetime(mc['month_year']).dt.year - mc['odp_date'].dt.year) * 12 + pd.to_datetime(mc['month_year']).dt.month - mc['odp_date'].dt.month

In [59]:
mc['policy'] = np.where(mc['time']< 0, 0, 1)

In [60]:

mc['treatment'] = 1

In [61]:
mc.head()

,month_year,count,city,state,odp_date,year,time,policy,treatment
0,2015-12,13,Riverside city,CA,2017-03-07,2015,-15,0,1
1,2016-01,47,Riverside city,CA,2017-03-07,2016,-14,0,1
2,2016-02,49,Riverside city,CA,2017-03-07,2016,-13,0,1
3,2016-03,43,Riverside city,CA,2017-03-07,2016,-12,0,1
4,2016-04,39,Riverside city,CA,2017-03-07,2016,-11,0,1


In [97]:
cd = pd.read_csv('city_data.csv')

In [19]:
cd.head()

,NAME,population,total_hh,pct_25_34,pct_hispanic,pct_white,pct_black,median_age,gross_rent,median_hh_income,pct_bachelor,state,place,year,avg_hh_size,city,state_name
0,"Birmingham city, Alabama",213434,90117.0,17.2,3.6,24.6,71.3,36.0,777,36241,16.9,1,7000,2016,2.368410,Birmingham city,Alabama
1,"Dothan city, Alabama",67714,26119.0,13.6,2.9,60.9,35.0,38.9,720,42336,14.7,1,21184,2016,2.592519,Dothan city,Alabama
2,"Hoover city, Alabama",84943,32110.0,15.3,7.6,70.9,16.7,36.6,1021,79004,34.1,1,35896,2016,2.645375,Hoover city,Alabama
3,"Huntsville city, Alabama",196225,80540.0,14.3,6.3,60.6,31.5,35.6,766,51895,25.7,1,37000,2016,2.436367,Huntsville city,Alabama
4,"Mobile city, Alabama",192895,79188.0,15.8,2.5,46.4,49.8,36.9,804,39463,18.9,1,50000,2016,2.435912,Mobile city,Alabama


In [98]:
mc['city_state'] = mc['city'] + mc['state']
cd['city_state'] = cd['city'] + cd['state_code']
mc_mg = mc.merge(cd, how ='left', on = 'city_state')

In [99]:
mc_mg = mc_mg.merge(cities, how= 'left', left_on = 'city_x', right_on = 'city')

In [100]:
# create year dummies
#mc_mg['2013'] = np.where(mc_mg['year_x']== 2013, 1, 0)
#mc_mg['2014'] = np.where(mc_mg['year_x']== 2014, 1, 0)
mc_mg['2015'] = np.where(mc_mg['year_x']== 2015, 1, 0)
mc_mg['2016'] = np.where(mc_mg['year_x']== 2016, 1, 0)
mc_mg['2017'] = np.where(mc_mg['year_x']== 2017, 1, 0)
mc_mg['2018'] = np.where(mc_mg['year_x']== 2018, 1, 0)

In [101]:
# create time since policy dummies
#mc_mg['0_6mos_odp'] = np.where((mc_mg['time']>= 0) & (mc_mg['time']<6), 1, 0)
#mc_mg['6_12mos_odp'] = np.where((mc_mg['time']>= 6) & (mc_mg['time']<12), 1, 0)
#mc_mg['12_18mos_odp'] = np.where((mc_mg['time']>= 12) & (mc_mg['time']<18), 1, 0)
#mc_mg['18_24mos_odp'] = np.where((mc_mg['time']>= 18) & (mc_mg['time']<24), 1, 0)
#mc_mg['over_24mos_odp'] = np.where(mc_mg['time']>= 24, 1, 0)

In [102]:
# create governance dummies
mc_mg['mayor_council'] = np.where(mc_mg['Governance']== 'Mayor-Council', 1, 0)
mc_mg['council_manager'] = np.where(mc_mg['Governance']== 'Council-Manager', 1, 0)
#mc_mg['mayor_commission'] = np.where(mc_mg['Governance']== 'Mayor-Commission', 1, 0)
#mc_mg['commissioner_manager'] = np.where(mc_mg['Governance']== 'Commissioner-Manager', 1, 0)
#mc_mg['other'] = np.where((mc_mg['Governance']!= 'Council-Manager') & (mc_mg['Governance']!= 'Mayor-Council'), 1, 0)

In [103]:
# create state political lean dummies
#mc_mg['competitive'] = np.where(mc_mg['Pol_Lean_State']== 'Competitive', 1, 0)
#mc_mg['lean_dem'] = np.where(mc_mg['Pol_Lean_State']== 'Lean Democratic', 1, 0)
#mc_mg['solid_dem'] = np.where(mc_mg['Pol_Lean_State']== 'Solid Democratic', 1, 0)
#mc_mg['lean_rep'] = np.where(mc_mg['Pol_Lean_State']== 'Lean Republican', 1, 0)
#mc_mg['solid_rep'] = np.where(mc_mg['Pol_Lean_State']== 'Solid Republicans', 1, 0)

mc_mg['rep'] = np.where((mc_mg['Pol_Lean_State']== 'Lean Republican') | (mc_mg['Pol_Lean_State']== 'Solid Republican'), 1, 0)
mc_mg['dem'] = np.where((mc_mg['Pol_Lean_State']== 'Lean Democrat') | (mc_mg['Pol_Lean_State']== 'Solid Democrat'), 1, 0)


In [ ]:
# create interaction effect variables

mc_mg['pop_treat'] = mc_mg['treatulation'] * mc_mg['policy']
mc_mg['age_treat'] = mc_mg['median_age'] * mc_mg['policy']
mc_mg['rent_treat'] = mc_mg['median_gross_rent'] * mc_mg['policy']
mc_mg['income_treat'] = mc_mg['median_income'] * mc_mg['policy']
mc_mg['25_treat'] = mc_mg['pct_25_34'] * mc_mg['policy']
mc_mg['male_treat'] = mc_mg['pct_male'] * mc_mg['policy']
mc_mg['black_treat'] = mc_mg['pct_black'] * mc_mg['policy']
mc_mg['bachelor_treat'] = mc_mg['pct_bachelor'] * mc_mg['policy']
mc_mg['budget_treat'] = mc_mg['Budget_1617'] * mc_mg['policy']
mc_mg['2015_treat'] = mc_mg['2015'] * mc_mg['policy']
mc_mg['2016_treat'] = mc_mg['2016'] * mc_mg['policy']
mc_mg['2017_treat'] = mc_mg['2017'] * mc_mg['policy']
mc_mg['2018_treat'] = mc_mg['2018'] * mc_mg['policy']
mc_mg['mayor_treat'] = mc_mg['mayor_council'] * mc_mg['policy']
mc_mg['manager_treat'] = mc_mg['council_manager'] * mc_mg['policy']
mc_mg['dem_treat'] = mc_mg['dem'] * mc_mg['policy']
mc_mg['rep_treat'] = mc_mg['rep'] * mc_mg['policy']
mc_mg['time_treat'] = mc_mg['time'] * mc_mg['policy']

In [107]:
riverside = mc_mg[['month_year', 'count', 'city', 'state_x', 'time', 'policy', 'treatment', 'population',
                   'median_age', 'median_gross_rent', 'median_income', 'pct_25_34', 'pct_male',
                  'pct_black', 'pct_white', 'pct_bachelor', 'Budget_1617', '2015',
                  '2016', '2017', '2018','mayor_council', 'council_manager', 'dem', 'rep']]

In [108]:
riverside.to_csv('riverside_clean.csv', index = False)